### Installation

In [ ]:
!pip install unsloth
!pip install scikit-learn
!pip install wandb -qqq

### Unsloth

In [ ]:
import wandb
wandb.login()

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

  2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

  ········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gauravganju (gauravganju-srm-institute-of-science-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.3-70B-Instruct-bnb-4bit", # choose "unsloth/Llama-3.2-1B-Instruct" or "unsloth/Llama-3.3-70B-Instruct-bnb-4bit"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.3: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA A100 80GB PCIe. Num GPUs = 1. Max memory: 79.138 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2026.1.3 patched 80 layers with 80 QKV layers, 80 O layers and 80 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [6]:
wandb login

wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gauravganju (gauravganju-srm-institute-of-science-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.3",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

from datasets import load_dataset
dataset = load_dataset("Frenzyknight/clarity-cot-dataset", split = "train")

train/train.parquet:   0%|          | 0.00/4.51M [00:00<?, ?B/s]

test/test.parquet:   0%|          | 0.00/416k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3448 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/308 [00:00<?, ? examples/s]

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [6]:
# from unsloth.chat_templates import standardize_sharegpt
# dataset = standardize_sharegpt(dataset)
# we are already using HuggingFace's generic format
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/3448 [00:00<?, ? examples/s]

We look at how the conversations are structured for item 5:

In [ ]:
dataset[5]["conversations"]

[{'content': "You are an expert political discourse analyst specializing in classifying response clarity in political interviews. Your task is to determine whether a politician's response to a specific question is a Clear Reply, Clear Non-Reply, or Ambivalent.",
  'role': 'system'},
 {'content': "Based on a segment of the interview where the interviewer asks a series of questions, classify the type of response provided by the interviewee for the following question.\n\n                ### Classification Categories ###\n                1. Clear Reply - The information requested is explicitly stated (in the requested form)\n                2. Clear Non-Reply - The information requested is not given at all due to ignorance, need for clarification, or declining to answer\n                3. Ambivalent - The information requested is given in an incomplete way (e.g., the answer is too general, partial, implicit, dodging, or deflection)\n\n                ### Full Interview Question ###\n     

And we see how the chat template transformed these conversations.

**[Notice]** Llama 3.1 Instruct's default chat template default adds `"Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024"`, so do not be alarmed!

In [ ]:
dataset[5]["text"]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are an expert political discourse analyst specializing in classifying response clarity in political interviews. Your task is to determine whether a politician's response to a specific question is a Clear Reply, Clear Non-Reply, or Ambivalent.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nBased on a segment of the interview where the interviewer asks a series of questions, classify the type of response provided by the interviewee for the following question.\n\n                ### Classification Categories ###\n                1. Clear Reply - The information requested is explicitly stated (in the requested form)\n                2. Clear Non-Reply - The information requested is not given at all due to ignorance, need for clarification, or declining to answer\n                3. Ambivalent - The information requested is given in an incomplete way (

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [8]:
from trl import SFTConfig, SFTTrainer
from transformers import DataCollatorForSeq2Seq
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 2, # Set this for 1 full training run.
        # max_steps = 5,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "wandb", # Use TrackIO/WandB etc
    ),
)

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [9]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=64):   0%|          | 0/3448 [00:00<?, ? examples/s]

We verify masking is actually done:

In [10]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

"<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are an expert political discourse analyst specializing in classifying response clarity in political interviews. Your task is to determine whether a politician's response to a specific question is a Clear Reply, Clear Non-Reply, or Ambivalent.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nBased on a segment of the interview where the interviewer asks a series of questions, classify the type of response provided by the interviewee for the following question.\n\n                ### Classification Categories ###\n                1. Clear Reply - The information requested is explicitly stated (in the requested form)\n                2. Clear Non-Reply - The information requested is not given at all due to ignorance, need for clarification, or declining to answer\n                3. Ambivalent - The information requested is given in an

In [11]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

We can see the System and Instruction prompts are successfully masked!

In [12]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100 80GB PCIe. Max memory = 79.138 GB.
39.221 GB of memory reserved.


In [13]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,448 | Num Epochs = 2 | Total steps = 862
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 207,093,760 of 70,760,800,256 (0.29% trained)


wandb: Detected [huggingface_hub.inference] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.594600
2,1.816800
3,0.000200
4,2.637400
5,3.220100
6,0.086100
7,0.459400
8,0.550000
9,0.692700
10,0.175200


train/epoch,▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇█
train/global_step,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇██
train/grad_norm,▄▆▃▆▂▂▂▂▃▂▃▂▃▄▃▃▃▃▄▃█▂▅▂▂▃▄▃▄▂▂▁▂▂▁▂▂▃▃▃
train/learning_rate,█████▇▇▇▆▆▆▆▆▅▅▅▅▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▂▂▁▁▁▁▁
train/loss,█▁▁▁▁▁▁▁▂▁▁▁▁▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁
total_flos,2.6530229981948805e+18
train/epoch,2
train/global_step,862
train/grad_norm,0.67095
train/learning_rate,0.0
train/loss,0.1182


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

446.5262 seconds used for training.
7.44 minutes used for training.
Peak reserved memory = 6.531 GB.
Peak reserved memory for training = 3.896 GB.
Peak reserved memory % of max memory = 44.284 %.
Peak reserved memory for training % of max memory = 26.417 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!



We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [ ]:
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel
from datasets import load_dataset
import csv
import torch
import re

# 1. Apply LLaMA-3.3 chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.3",
)

# 2. Enable fast inference
FastLanguageModel.for_inference(model)

# 3. Load YOUR test dataset
dataset = load_dataset("Frenzyknight/clarity-dataset", split="test")
print(f"Loaded {len(dataset)} test examples")

# 4. Helper function to extract label from COT response
def extract_label(response):
    """Extract the final label from COT response."""
    # Look for "LABEL: <label>" pattern at the end
    match = re.search(r'LABEL:\s*(Clear Reply|Clear Non-Reply|Ambivalent)', response, re.IGNORECASE)
    if match:
        label = match.group(1)
        # Normalize capitalization
        label_map = {
            "clear reply": "Clear Reply",
            "clear non-reply": "Clear Non-Reply",
            "ambivalent": "Ambivalent",
        }
        return label_map.get(label.lower(), label)
    
    # Fallback: check if any label appears in last 50 chars
    last_part = response[-50:].lower()
    if "clear reply" in last_part and "non-reply" not in last_part:
        return "Clear Reply"
    elif "clear non-reply" in last_part or "non-reply" in last_part:
        return "Clear Non-Reply"
    elif "ambivalent" in last_part:
        return "Ambivalent"
    
    return "PARSE_ERROR"  # Couldn't extract label

# 5. Store all predictions
results = []

# 6. Loop through ALL test examples
for idx, example in enumerate(dataset):
    messages = example["conversations"]
    
    # Tokenize with generation prompt
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")
    
    # Generate prediction - INCREASED max_new_tokens for COT
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=2048,  # ⬆️ Increased from 10 for COT reasoning
        temperature=0.1,
        use_cache=True,
    )
    
    # Extract full response
    prompt_len = inputs.shape[-1]
    full_response = tokenizer.batch_decode(
        outputs[:, prompt_len:],
        skip_special_tokens=True,
    )[0].strip()
    
    # Extract just the label
    prediction = extract_label(full_response)
    
    results.append({
        "prediction": prediction,
        "full_response": full_response,  # Optional: save full COT for analysis
    })
    
    # Progress indicator
    if (idx + 1) % 50 == 0:
        print(f"Processed {idx + 1}/{len(dataset)} examples")
    
    # Free up GPU memory periodically
    if (idx + 1) % 100 == 0:
        torch.cuda.empty_cache()

# 7. Save predictions to CSV
with open("clarity_predictions_cot.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["prediction", "full_response"])
    writer.writeheader()
    writer.writerows(results)

print(f"\n✅ All {len(results)} predictions saved to clarity_predictions_cot.csv")

# Check for parse errors
parse_errors = sum(1 for r in results if r["prediction"] == "PARSE_ERROR")
if parse_errors > 0:
    print(f"⚠️  {parse_errors} responses couldn't be parsed - check full_response column")

print(f"\nSample predictions:")
for i in range(min(3, len(results))):
    print(f"\n--- Example {i+1} ---")
    print(f"Label: {results[i]['prediction']}")
    print(f"Reasoning (last 200 chars): ...{results[i]['full_response'][-200:]}")

Loaded 308 test examples

MESSAGES BEING SENT TO MODEL:
Number of messages: 2

[Message 0] Role: system
Content (first 300 chars): You are an expert political discourse analyst specializing in classifying response clarity in political interviews. Your task is to determine whether a politician's response to a specific question is a Clear Reply, Clear Non-Reply, or Ambivalent....

[Message 1] Role: user
Content (first 300 chars): Based on a segment of the interview where the interviewer asks a series of questions, classify the type of response provided by the interviewee for the following question.

                ### Classification Categories ###
                1. Clear Reply - The information requested is explicitly stat...


FORMATTED PROMPT (With Chat Template Applied):
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are an expert political discourse analyst specializing in classifying response clarity

In [28]:
print(results[0])

{'prediction': 'Ambivalent'}


In [ ]:
from datasets import load_dataset
dataset = load_dataset("ailsntua/QEvasion", split = "test")
dataset[0]

{'title': None,
 'date': None,
 'president': None,
 'url': 'https://www.presidency.ucsb.edu/documents/the-presidents-news-conference-3',
 'question_order': 5,
 'interview_question': 'Q. What about the redline, sir?',
 'interview_answer': "Well, the world has made it clear that these tests caused us to come together and work in the United Nations to send a clear message to the North Korean regime. We're bound up together with a common strategy to solve this issue peacefully through diplomatic means.Kevin [Kevin Corke, NBC News].",
 'gpt3.5_summary': None,
 'gpt3.5_prediction': None,
 'question': ' Inquiring about the status or information regarding the redline.',
 'annotator_id': None,
 'annotator1': 'Dodging',
 'annotator2': 'General',
 'annotator3': 'Dodging',
 'inaudible': False,
 'multiple_questions': False,
 'affirmative_questions': True,
 'index': 0,
 'clarity_label': 'Ambivalent',
 'evasion_label': ''}

In [ ]:
import pandas as pd
from datasets import load_dataset

# Load CSV
df = pd.read_csv("clarity_predictions_cot.csv")

# Load dataset
dataset = load_dataset("ailsntua/QEvasion", split="test")

# Add columns from dataset
df["clarity_label"] = [row["clarity_label"] for row in dataset]
df["annotator1"] = [row["annotator1"] for row in dataset]
df["annotator2"] = [row["annotator2"] for row in dataset]
df["annotator3"] = [row["annotator3"] for row in dataset]

# Save updated CSV
df.to_csv("clarity_predictions_with_labels.csv", index=False)

print("Updated CSV saved as clarity_predictions_with_labels.csv")

Updated CSV saved as clarity_predictions_with_labels.csv


In [37]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score

# Load CSV
df = pd.read_csv("clarity_predictions_with_labels.csv")

# Keep only required columns
y_true = df["clarity_label"].str.strip()
y_pred = df["prediction"].str.strip()

# Optional: enforce valid label set
VALID = {"Clear Reply", "Clear Non-Reply", "Ambivalent"}
mask = y_true.isin(VALID) & y_pred.isin(VALID)

y_true = y_true[mask]
y_pred = y_pred[mask]


In [38]:
# Accuracy
accuracy = accuracy_score(y_true, y_pred)

# Precision / Recall / F1 (Macro)
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
    y_true,
    y_pred,
    average="macro"
)

# 🔹 NEW: Weighted F1 (paper Eq. 2)
f1_weighted = f1_score(
    y_true,
    y_pred,
    average="weighted"
)

print("=== Evaluation Metrics ===")
print(f"Accuracy        : {accuracy:.4f}")
print(f"Precision (Mac) : {precision_macro:.4f}")
print(f"Recall (Mac)    : {recall_macro:.4f}")
print(f"F1 (Macro)     : {f1_macro:.4f}")
print(f"F1 (Weighted)  : {f1_weighted:.4f}")


=== Evaluation Metrics ===
Accuracy        : 0.8214
Precision (Mac) : 0.7994
Recall (Mac)    : 0.7457
F1 (Macro)     : 0.7669
F1 (Weighted)  : 0.8211


In [41]:
from sklearn.metrics import classification_report
# Detailed per-class report
print(
    classification_report(
        y_true,
        y_pred,
        digits=4
    )
)

                 precision    recall  f1-score   support

     Ambivalent     0.8689    0.8689    0.8689       206
Clear Non-Reply     0.8235    0.6087    0.7000        23
    Clear Reply     0.7059    0.7595    0.7317        79

       accuracy                         0.8214       308
      macro avg     0.7994    0.7457    0.7669       308
   weighted avg     0.8237    0.8214    0.8211       308



### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
model.push_to_hub_merged("Frenzyknight/Clarity-llama-70b", tokenizer, save_method = "merged_16bit", token = "HF_Token")

config.json:   0%|          | 0.00/891 [00:00<?, ?B/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Found HuggingFace hub cache directory: /workspace/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

Unsloth: Merging weights into 16bit:  97%|█████████▋| 29/30 [36:14<01:17, 77.41s/it]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Unsloth: Merging weights into 16bit: 100%|██████████| 30/30 [36:43<00:00, 73.44s/it]


Unsloth: Merge process complete. Saved to `/workspace/Frenzyknight/Clarity-llama-70b`
